In [1]:
import os
import cv2
from matplotlib import pyplot as plt
import random
import pandas as pd
import numpy as np
import uuid

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
#setting gpu growth
gpu = tf.config.experimental.list_physical_devices('GPU')
for g in gpu:
    tf.config.experimental.set_memory_growth(g, True)

In [8]:
pospath = os.path.join('data', 'positive')
negpath = os.path.join('data', 'negative')
anchor = os.path.join('data', 'anchor')

In [7]:
cap  = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[80:250+80, 200:250+200, :]    
    cv2.imshow('image', frame)
    if cv2.waitKey(1) & 0XFF == ord('a'):
        img = os.path.join(anchor, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(img, frame)
        
    if cv2.waitKey(1) & 0XFF == ord('p'):
        img = os.path.join(pospath, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(img, frame)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()    

In [6]:
os.makedirs(pospath)
os.makedirs(negpath)
os.makedirs(anchor)

In [9]:
for directories in os.listdir("lfw"):
    for files in os.listdir(os.path.join('lfw',directories)):
        ex = os.path.join("lfw", directories, files)
        new = os.path.join(negpath, files)
        os.replace(ex, new)